# INSERT INTO DB

In [1]:
import pandas as pd
import numpy as np

import warnings
import os
warnings.filterwarnings(action='ignore')

items = pd.read_json('./items_prc.json')
reviews = pd.read_json('./reviews_prc_1.json')



In [32]:
reviews = reviews.replace({np.nan: None})

In [22]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 860 entries, 0 to 859
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   item_id  860 non-null    int64 
 1   cat      860 non-null    object
 2   name     860 non-null    object
 3   price    860 non-null    int64 
 4   img      860 non-null    object
 5   link     860 non-null    object
dtypes: int64(2), object(4)
memory usage: 40.4+ KB


In [79]:
import sqlite3
conn = sqlite3.connect('./apparel.db')
cur = conn.cursor()

### item table

In [8]:
# 데이터 입력에 필요한 형태로 변환
items = items[['item_id', 'cat', 'name', 'price', 'img', 'link']]
items.item_id = items.item_id.astype(str)

In [9]:
for row in items.itertuples(index=False, name=None):
    cur.execute('''
                INSERT INTO item VALUES (?, ?, ?, ?, ?, ?); ''', row)
    conn.commit()

In [60]:
reviews['사이즈'].unique()

array(['2', '3', None, '1', '4', 'F'], dtype=object)

## reviews => user, review로 나누기

In [39]:
reviews.head()

,품번,아이디,별점,키,몸무게,평소사이즈,리뷰,사이즈,색상
0,0,핫핑-지원,-1,161~165,65~,3.0,지금 제일 핫하다는 오바오 원피스!?🐼\n전참시 보다가 오바오님이 입고 계시는 원피...,2,블랙
1,0,mozzi,5,161~165,65~,4.0,요즘은 맨날 롱원피스만 샀는데 어쩔수 없이 너무 더울 땐 힘들어서 짧은 거 사봤어요...,3,블랙
2,0,183019735,5,151~155,65~,4.0,실물이 훨씬 예뻐요 키가 154 인 저한테는 길이가 더 길긴했지만 너무 안짧아서 좋...,3,블랙
3,0,solve,5,None,None,NaN,레전드에요 ㅋㅋ 이 옷은 그닥 기대도 안하고 샀는데\n착용감이최고 ㅜㅜ\n일단 너무...,3,소라
4,0,7481078,5,166~170,65~,4.0,배송 빨랐어요 주문한 다음날 출발해서 그다음날 바로 받았어요 ㅎㅎ\n옷은 화면과같고...,3,블랙


In [66]:
# user, review 정보 나누기
user = reviews[['아이디', '키', '몸무게', '평소사이즈']]
review = reviews[['아이디', '품번', '별점', '리뷰', '사이즈', '색상']]

### user Table

In [37]:
for col in ['키', '몸무게', '평소사이즈']:
    print(col)
    print(user[col].unique())

키
['161~165' '151~155' None '166~170' '156~160' '171~175' '146~150' '176이상'
 '140~145' '139이하']
몸무게
['65~' None '56 ~ 60' '61 ~ 65' '51 ~ 55' '46 ~ 50' '~45']
평소사이즈
[3.0 4.0 None 2.0 1.0 5.0]


In [47]:
user_dup = user.loc[user['아이디'].duplicated()].index
user.drop(user_dup, axis=0, inplace=True)
user.reset_index(drop=True, inplace=True)
user.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40398 entries, 0 to 40397
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   아이디     40398 non-null  object
 1   키       30206 non-null  object
 2   몸무게     27638 non-null  object
 3   평소사이즈   27409 non-null  object
dtypes: object(4)
memory usage: 1.2+ MB


In [58]:
# 아이디 중복 확인 후 데이터 입력
for row in user.itertuples(name=None, index=False):
    # cur.execute(f'''
    #             INSERT INTO user VALUES(?, ?, ?, ?)
    #             SELECT * FROM user
    #             WHERE NOT EXISTS
    #             (SELECT 1 FROM USER
    #             WHERE id == {row[0]});''', row)
    cur.execute('''
                INSERT OR IGNORE
                INTO user VALUES(?,?,?,?);''', row) # 아이디 중복 제외
    
    conn.commit()

In [59]:
cur.execute('''
            SELECT COUNT(*) FROM user;''')
print(cur.fetchone())

(40398,)


### review Table

In [73]:
review = review[['아이디', '품번', '별점', '리뷰', '사이즈', '색상']]
review_dup = review.loc[review['리뷰'].duplicated()==True].index
review.drop(review_dup, axis=0, inplace=True)
review.reset_index(drop=True, inplace=True)
len(review)

78588

In [74]:
review.duplicated().sum()

0

In [80]:
for row in review.itertuples(name=None, index=True):
    cur.execute('''
                INSERT OR IGNORE
                INTO REVIEW VALUES(?,?,?,?,?,?,?);''', row)
    conn.commit()

## 연결 해제

In [81]:
cur.close()
conn.close()